In [76]:
import numpy as np
import keras
from keras.datasets import cifar10
from keras.layers import Dense, Conv2D, Flatten, Activation, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.utils import np_utils


In [77]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
n_examples = 5000

In [78]:
X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

for ix in range(n_examples):
    if y_train[ix] < 5:
        # put data in set 1
        X1_train.append(x_train[ix]/255.0)
        Y1_train.append(y_train[ix])
    else:
        # put data in set 2
        X2_train.append(x_train[ix]/255.0)
        Y2_train.append(y_train[ix])

for ix in range(y_test.shape[0]):
    if y_test[ix] < 5:
        # put data in set 1
        X1_test.append(x_test[ix]/255.0)
        Y1_test.append(y_test[ix])
    else:
        # put data in set 2
        X2_test.append(x_test[ix]/255.0)
        Y2_test.append(y_test[ix])


In [79]:
#Convert the input to an array
X1_train = np.asarray(X1_train).reshape((-1, 32, 32, 3))
X1_test = np.asarray(X1_test).reshape((-1, 32, 32, 3))
X2_train = np.asarray(X2_train).reshape((-1, 32, 32, 3))
X2_test = np.asarray(X2_test).reshape((-1, 32, 32, 3))

# One hot representation for multiclass problem
# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y1_train = np_utils.to_categorical(np.asarray(Y1_train), num_classes=5)
Y1_test = np_utils.to_categorical(np.asarray(Y1_test), num_classes=5)

Y2_train = np_utils.to_categorical(np.asarray(Y2_train), num_classes=10)
Y2_test = np_utils.to_categorical(np.asarray(Y2_test), num_classes=10)

In [80]:
print (X1_train.shape, X1_test.shape)
print (Y1_train.shape, Y1_test.shape)


(2489, 32, 32, 3) (5000, 32, 32, 3)
(2489, 5) (5000, 5)


In [81]:
split1 = int(0.8 * X1_train.shape[0])
split2 = int(0.8 * X2_train.shape[0])

x1_val = X1_train[split1:]
x1_train = X1_train[:split1]
y1_val = Y1_train[split1:]
y1_train = Y1_train[:split1]

x2_val = X2_train[split2:]
x2_train = X2_train[:split2]
y2_val = Y2_train[split2:]
y2_train = Y2_train[:split2]

http://cs231n.github.io/assets/conv-demo/index.html

In [87]:
model = Sequential()

model.add(Conv2D(32, (5, 5), input_shape=(32, 32, 3), activation='relu'))
model.add(Conv2D(16, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dropout(0.42))

model.add(Dense(128, activation='relu'))
model.add(Dense(5,activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 24, 24, 16)        12816     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 10, 10, 8)         1160      
_________________________________________________________________
flatten_10 (Flatten)         (None, 800)               0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 800)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 128)               102528    
__________

In [88]:
import time, datetime

start = datetime.datetime.now()
hist1 = model.fit(x1_train, y1_train,
                  batch_size=100,
                  epochs=10,
                  verbose=2,
                  shuffle=True)

time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for first training: ', time_taken)
print ('\n', '-'*20, '\n'*2)

Epoch 1/10
5s - loss: 1.5818 - acc: 0.2732
Epoch 2/10
4s - loss: 1.4384 - acc: 0.3968
Epoch 3/10
4s - loss: 1.2563 - acc: 0.4802
Epoch 4/10
4s - loss: 1.1560 - acc: 0.5143
Epoch 5/10
4s - loss: 1.1184 - acc: 0.5274
Epoch 6/10
4s - loss: 1.0573 - acc: 0.5545
Epoch 7/10
4s - loss: 1.0006 - acc: 0.5892
Epoch 8/10
4s - loss: 0.9789 - acc: 0.5997
Epoch 9/10
4s - loss: 0.9467 - acc: 0.6067
Epoch 10/10
4s - loss: 0.9555 - acc: 0.5982


 -------------------- 

Time taken for first training:  0:00:44.691678

 -------------------- 




In [43]:
for l in model.layers[:6]:
    l.trainable = False  

In [44]:
trans_model = Sequential(model.layers[:6])

trans_model.add(Dense(128))
trans_model.add(Activation('relu'))
trans_model.add(Dense(10))
trans_model.add(Activation('softmax'))

trans_model.summary()
trans_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 24, 24, 16)        12816     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 10, 10, 8)         1160      
_________________________________________________________________
flatten_6 (Flatten)          (None, 800)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               102528    
__________

In [45]:
start = datetime.datetime.now()
hist2 = trans_model.fit(x2_train, y2_train, epochs=10, shuffle=True, batch_size=100, verbose=2)
time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for final training: ', time_taken)
print ('\n', '-'*20, '\n'*2)

Epoch 1/10
2s - loss: 1.6549 - acc: 0.4253
Epoch 2/10
2s - loss: 1.0703 - acc: 0.5931
Epoch 3/10
2s - loss: 0.9708 - acc: 0.6429
Epoch 4/10
2s - loss: 0.9490 - acc: 0.6524
Epoch 5/10
2s - loss: 0.9261 - acc: 0.6539
Epoch 6/10
2s - loss: 0.8822 - acc: 0.6643
Epoch 7/10
2s - loss: 0.8764 - acc: 0.6668
Epoch 8/10
2s - loss: 0.8583 - acc: 0.6743
Epoch 9/10
2s - loss: 0.8298 - acc: 0.6972
Epoch 10/10
2s - loss: 0.8342 - acc: 0.6873


 -------------------- 

Time taken for final training:  0:00:22.914861

 -------------------- 


